In [1]:
import csv
import monpa
from monpa import utils
from collections import Counter

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [2]:
tf_counter = Counter() #統計 tf 用
df_counter = Counter() #統計 df 用

result = list() #儲存處理過的結果
f = open('./limit_up.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
i = 0
for row in rows: #對內容逐行列舉，row[0]是title, row[1]是content
    i = i+1 

    if(i > 1): #略過第一行:欄位名稱
        query = row[0]+' '+row[1]
        df_tmp = Counter() #暫存本篇 df 用
        str='' #暫存本篇斷詞後的內容用
        sentence_list = utils.short_sentence(query) #斷句
        for item in sentence_list:
            result_cut = monpa.cut(item) #斷詞
            for term in result_cut:
                term = term.strip() #去除前後多餘空白
                if(len(term) > 1): #若詞長>1
                    str=str+' '+term
                    tf_counter[term] += 1 #tf加1
                    if(df_tmp[term] == 0): #若本篇之前不曾出現
                        df_tmp[term]=1 #df標為1
    #     print(i, str)
        result.append(str)
        df_counter += df_tmp # 累加多篇df

f.close() #關閉檔案
#print(tf_counter) #這個字 在所有文章 出現幾次
#print(df_counter) #這個字 在多少篇文章有出現

/Users/peifish/opt/anaconda3/lib/python3.7/site-packages/monpa/crf_layer.py:374: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/TensorCompare.cpp:333.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


In [3]:
total = 0
for word in tf_counter:
    total += 1
print("dictionary size: ",total, "words")

dictionary size:  15250 words


In [4]:
for term in df_counter.most_common(100): #印出df前100名
    print("[",term[0],term[1],"]")

[ 漲停 580 ]
[ 股價 447 ]
[ 持續 334 ]
[ 台北 321 ]
[ 營收 318 ]
[ 今年 311 ]
[ 記者 309 ]
[ 市場 271 ]
[ 報導 264 ]
[ 時報 262 ]
[ 新高 257 ]
[ 成長 240 ]
[ 目前 236 ]
[ 今日 233 ]
[ 公司 232 ]
[ 需求 226 ]
[ 去年 211 ]
[ 產品 206 ]
[ 營運 206 ]
[ 表示 205 ]
[ 表現 198 ]
[ 獲利 197 ]
[ 帶動 183 ]
[ 10 182 ]
[ 可望 176 ]
[ 由於 175 ]
[ 加上 168 ]
[ 20 154 ]
[ 走高 153 ]
[ 2021年 152 ]
[ 年增 152 ]
[ 漲幅 151 ]
[ 攻上 151 ]
[ 法人 148 ]
[ 上漲 148 ]
[ 預期 143 ]
[ 受惠 143 ]
[ 今天 143 ]
[ 開高 143 ]
[ 大漲 141 ]
[ 強勁 140 ]
[ 增加 139 ]
[ 歷史 138 ]
[ 開盤 137 ]
[ 維持 137 ]
[ 財訊 137 ]
[ 焦點股 135 ]
[ 客戶 134 ]
[ 研究員 133 ]
[ 早盤 131 ]
[ 業績 127 ]
[ 短線 127 ]
[ 指出 126 ]
[ 看好 125 ]
[ 跳空 124 ]
[ 預估 123 ]
[ 價格 122 ]
[ 指標 122 ]
[ 快報 122 ]
[ 累計 119 ]
[ 明年 118 ]
[ 預計 117 ]
[ 創下 116 ]
[ 訂單 116 ]
[ 全球 116 ]
[ 以來 115 ]
[ 疫情 115 ]
[ 超過 115 ]
[ 展望 114 ]
[ 主要 113 ]
[ 未來 113 ]
[ 淨利 112 ]
[ EPS 111 ]
[ 對於 109 ]
[ 指數 108 ]
[ 產能 108 ]
[ 其中 106 ]
[ 有利 106 ]
[ KD 106 ]
[ 出貨 105 ]
[ 合併 105 ]
[ 亮燈 105 ]
[ 支撐 104 ]
[ 上半年 103 ]
[ 以及 102 ]
[ 相關 102 ]
[ 半導體 102 ]
[ 開始 101 ]
[ 台灣 101 ]
[ 族群 99 ]
[ 以

In [5]:
#檢驗是否是中文或英文字符
def is_all_chinese(strs):
    for _char in strs:
        if ('\u4e00' <= _char <= '\u9fa5') or (u'\u0041'<= _char <= u'\u005a') or (u'\u0061'<= _char <= u'\u007a'):
            return True
    return False

In [6]:
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords

In [7]:
def hasNumber(stringVal):
    return any(elem.isdigit() for elem in stringVal)

In [8]:
data = df_counter.items()
sorted_data = sorted(data, key=lambda tup: tup[1], reverse=True)

stopwords = stopwordslist('./stopwords.txt')

rank = []
in_stops = []
num_stops = []
for tup in sorted_data:
    if tup[0] in stopwords:
        in_stops.append(tup[0])
    elif hasNumber(tup[0]):
        num_stops.append(tup[0])
    else:
        rank.append(tup)
    if len(rank) == 100:
        break

print("Stopwords:", in_stops) 
print("number words:", num_stops) 
for i in range(100):
    print(i+1, rank[i])

Stopwords: ['由於', '加上', '對於', '其中', '以及', '開始', '另外', '隨著', '不過']
number words: ['10', '20', '2021年', '5月']
1 ('漲停', 580)
2 ('股價', 447)
3 ('持續', 334)
4 ('台北', 321)
5 ('營收', 318)
6 ('今年', 311)
7 ('記者', 309)
8 ('市場', 271)
9 ('報導', 264)
10 ('時報', 262)
11 ('新高', 257)
12 ('成長', 240)
13 ('目前', 236)
14 ('今日', 233)
15 ('公司', 232)
16 ('需求', 226)
17 ('去年', 211)
18 ('產品', 206)
19 ('營運', 206)
20 ('表示', 205)
21 ('表現', 198)
22 ('獲利', 197)
23 ('帶動', 183)
24 ('可望', 176)
25 ('走高', 153)
26 ('年增', 152)
27 ('漲幅', 151)
28 ('攻上', 151)
29 ('法人', 148)
30 ('上漲', 148)
31 ('預期', 143)
32 ('受惠', 143)
33 ('今天', 143)
34 ('開高', 143)
35 ('大漲', 141)
36 ('強勁', 140)
37 ('增加', 139)
38 ('歷史', 138)
39 ('開盤', 137)
40 ('維持', 137)
41 ('財訊', 137)
42 ('焦點股', 135)
43 ('客戶', 134)
44 ('研究員', 133)
45 ('早盤', 131)
46 ('業績', 127)
47 ('短線', 127)
48 ('指出', 126)
49 ('看好', 125)
50 ('跳空', 124)
51 ('預估', 123)
52 ('價格', 122)
53 ('指標', 122)
54 ('快報', 122)
55 ('累計', 119)
56 ('明年', 118)
57 ('預計', 117)
58 ('創下', 116)
59 ('訂單', 116)
60 ('全球', 116)

In [9]:
f = open('./limit_up_term.csv', 'w', newline='', encoding='utf-8') #以寫入模式開啟另一個檔
writer = csv.writer(f) #準備寫入
for term in tf_counter.most_common(100): #印出tf前100名
    writer.writerow(term) #寫入一行
f.close()